In [1]:
import mysql.connector
from mysql.connector import Error
import configparser
from datetime import datetime, timedelta
import calendar

In [2]:
def actualizar_saldo_semana(semana, año):
    # Leer credenciales del archivo de configuración config.ini
    config = configparser.ConfigParser()
    config.read('config.ini')
    
    # Establecer la conexión a la base de datos
    try:
        user = config['database']['user']
        password = config['database']['password']
        host = config['database']['host']
        database = config['database']['database']

        # Abrir la base de datos: establecer la conexión
        conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

        # crear un cursor
        cursor = conexion.cursor()

        # =====================================================================================================================
        
        # Obtener semana y año de la semana anterior
        if semana == 1:
            año_anterior = año - 1
            if calendar.isleap(año_anterior):
                semana_anterior = 53
            else:
                semana_anterior = 52
        else:
            semana_anterior = semana - 1
            año_anterior = año

        # Verificar si hay registros para la semana anterior
        cursor.execute("SELECT * FROM tbl_saldo_semana WHERE semana = %s AND anio = %s", (semana_anterior, año_anterior))
        existing_record = cursor.fetchone()
            
        if not existing_record:
            print("No hay registro de saldos para la semana anterior. Por favor, verifique los parámetros o corrija la situación")
            return
            
        # Obtener saldo_inicial de la semana anterior
        cursor.execute("SELECT saldo_final FROM tbl_saldo_semana WHERE semana = %s AND anio = %s", (semana_anterior, año_anterior))
        saldo_inicial = cursor.fetchone()
            
        if saldo_inicial:
            saldo_inicial = saldo_inicial[0]
        else:
            print("No hay registro de saldos para la semana anterior. Por favor, verifique los parámetros o corrija la situación")
            return
            
        # Obtener neto_mes
        cursor.execute("SELECT SUM(NETO) FROM tbl_transacciones WHERE SEMANA = %s AND ANIO = %s", (semana, año))
        neto_mes = cursor.fetchone()[0]
            
        if neto_mes is None:
            neto_mes = 0
            
        # Obtener saldo_final de la semana actual
        saldo_final = saldo_inicial + neto_mes
            
        # Actualizar o insertar registro de semana
        cursor.execute("SELECT * FROM tbl_saldo_semana WHERE semana = %s AND anio = %s", (semana, año))
        existing_record = cursor.fetchone()
            
        if existing_record:
            # Si existe un registro, actualizar los campos
            cursor.execute("UPDATE tbl_saldo_semana SET saldo_inicial = %s, neto_mes = %s, saldo_final = %s WHERE semana = %s AND anio = %s", (saldo_inicial, neto_mes, saldo_final, semana, año))
        else:
            # Si no existe un registro, insertar uno nuevo
            # Obtener el primer día de la semana (lunes) con día de la semana 1
            primer_dia_semana = datetime.strptime(f'{año}-W{semana}-1', '%Y-W%W-%w')

            # Obtener el último día de la semana (domingo)
            last_day_of_week = primer_dia_semana + timedelta(days=6)

            month = last_day_of_week.month
            cursor.execute("INSERT INTO tbl_saldo_semana (fecha, anio, mes, semana, saldo_inicial, neto_mes, saldo_final) VALUES (%s, %s, %s, %s, %s, %s, %s)", (last_day_of_week, año, month, semana, saldo_inicial, neto_mes, saldo_final))
            
        print("Registro actualizado o insertado correctamente")
        conexion.commit()
    
    except Error as e:
        print("Error al conectarse a MySQL:", e)
        
    finally:
        if conexion.is_connected():
            cursor.close()
            conexion.close()
            print("Conexión MySQL cerrada")

# Ejemplo de uso
actualizar_saldo_semana(4, 2024)



No hay registro de saldos para la semana anterior. Por favor, verifique los parámetros o corrija la situación
Conexión MySQL cerrada
